## User TimeLine Analysing

In [83]:
import tweepy
import pandas as pd
import dateutil.parser
import re

In [4]:
app_api_key = '-'
app_api_secret_key = '-'


auth = tweepy.AppAuthHandler(app_api_key, app_api_secret_key)

In [53]:
def get_tweets(username: str, count: int = 1000, remove_rt: bool = False):
    '''get tweets tweeted by specific user'''
    api = tweepy.API(
            auth,
            wait_on_rate_limit=True
        )

    tweets = tweepy.Cursor(
            api.user_timeline,
            screen_name = username,
            tweet_mode = 'extended',
            count = count
        ).items()
    
    tweets = [tweet._json for tweet in tweets]
    df = pd.json_normalize(tweets)
    if remove_rt:
        return df[~df['retweeted_status.id'].isna()]
    return df[df['retweeted_status.id'].isna()]


In [58]:
df = get_tweets('bmwindia', 1000, True)
print("Tweets from: ", df['created_at'].min(), df['created_at'].max())
print("Total tweets: ", len(df))

Tweets from:  Fri Aug 07 11:01:38 +0000 2020 Wed Sep 11 03:40:28 +0000 2019
Total tweets:  262


In [84]:
df_clean = df[['created_at', 'full_text']].reset_index()

In [85]:
def parse_date(doc):
    date = dateutil.parser.parse(doc)
    return str(date.date())

In [86]:
df_clean['created_at'] = df_clean['created_at'].map(parse_date)

In [87]:
df_clean.head()

,index,created_at,full_text
0,193,2021-10-11,RT @autox: Winning our coveted 'Performance Ca...
1,548,2021-06-27,RT @odmag: This is the mid-life update for the...
2,549,2021-06-27,RT @autox: Priced between 62.90 lakh &amp; 71....
3,559,2021-06-24,"RT @MotorBeam: 2021 BMW 5-Series Launched, Pri..."
4,560,2021-06-24,RT @autocarindiamag: No sooner has the BMW 5 S...
